# Part 1

In [1]:
filename, row = "inputs/12-15.txt", 2000000
with open(filename, "r") as f:
    data = f.read()

In [2]:
import re

In [3]:
sensors = []
for l in data.splitlines():
    x, y, bx, by = (int(v) for v in re.findall("-?\d+", l))
    d = abs(bx -x) + abs(by - y)
    sensors.append((x, y, bx, by, d))

In [4]:
def add_range(ranges, range_):
    x, y = range_
    ret = []
    for (rx, ry) in ranges:
        if rx > y or x > ry:
            ret.append((rx, ry))
        else:
            x = min(x, rx)
            y = max(y, ry)
    ret.append((x, y))
    return ret

In [5]:
ranges = []
row_stations = set()
for x, y, bx, by, d in sensors:
    for s in ((x, y), (by, by)):
        if s[1] == row:
            row_stations.add(s)
    x_range = d - abs(y - row)
    if x_range > 0:
        ranges = add_range(ranges, (x - x_range, x + x_range))

sum(y - x + 1 for x, y in ranges) - len(row_stations)

5403290

# Part 2

An inefficient (brute force) solution.

In [6]:
MIN_POS, MAX_POS = 0, 4000000

In [7]:
def scan_row(row, x0=MIN_POS, y0=MAX_POS):
    ranges = []
    for x, y, bx, by, d in sensors:
        x_range = d - abs(y - row)
        if x_range > 0:
            ranges = add_range(ranges, (x - x_range, x + x_range))

    if sum(min(y, y0) - max(x, x0) for x, y in ranges) < y0 - x0:
        dbx = max(y for _, y in ranges if y < y0) + 1
        return dbx*y0 + row

In [ ]:
"""
for row in range(4000001):
    tf = scan_row(row)
    if tf is not None:
        print(tf)
        break
"""

A slightly less inefficient linear programming solution.

In [8]:
import mip

In [9]:
def abs_value(model, x):
    HUGE = 1000000000
    absx = m.add_var(var_type=mip.INTEGER)
    sgn = m.add_var(var_type=mip.BINARY)
    model += absx <= x + HUGE * (1 - sgn)
    model += absx >= x - HUGE * (1 - sgn)
    model += absx <= -x + HUGE * sgn
    model += absx >= -x - HUGE * sgn
    return absx

m = mip.Model()
x = m.add_var("x", lb=MIN_POS, ub=MAX_POS, var_type=mip.INTEGER)
y = m.add_var("y", lb=MIN_POS, ub=MAX_POS, var_type=mip.INTEGER)
for sx, sy, *_, d in sensors:
    dx = abs_value(m, sx - x)
    dy = abs_value(m, sy - y)
    m += dx + dy >= d + 1
status = m.optimize()
tfx = int(x.x)
tfy = int(y.x)
tfx*MAX_POS + tfy

10291582906626